In [1]:
import json
import os

timestamps = [i * 0.02 for i in range(1500 + 1)]
replace = {
    'Aee.': 'A.',
    'Bee.': 'B.',
    'See.': 'C.',
    'Dee.': 'D.',
}

In [2]:
with open('tatabahasa.json') as fopen:
    d = json.load(fopen)
    
len(d)

1284

In [6]:
result = []

for i in range(len(d)):
    audio = os.path.join('tatabahasa-v2', f'{i}.mp3')
    align = os.path.join('tatabahasa-v2-alignment_alignment', f'{i}.json')
    
    if not os.path.exists(audio):
        continue
    
    with open(align) as fopen:
        align = json.load(fopen)
    
    merged_data = []
    for entry in align[1:]:
        if entry['text'] == '.' and merged_data:
            merged_data[-1]['text'] += '.'
            merged_data[-1]['end'] = entry['end']
        else:
            merged_data.append(entry)

    for i in range(len(merged_data)):
        merged_data[i]['text'] = replace.get(merged_data[i]['text'], merged_data[i]['text'])
        
    align = merged_data
        
    segments, temp = [], [align[0]]
    last_t = align[0]['end']
    for c in align[1:]:
        if (c['start'] - last_t) > 0.2:
            segments.append(temp)
            temp = []

        last_t = c['end']
        temp.append(c)

    if len(temp):
        segments.append(temp)

    ts = []
    for s in segments:
        start = min(timestamps, key=lambda t: abs(t - s[0]['start']))
        end = min(timestamps, key=lambda t: abs(t - s[-1]['end']))
        w = ' '.join([c['text'] for c in s])
        t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
        ts.append(t)

    ts = ''.join(ts)
    new_text = text = f"<|startoftranscript|><|ms|><|transcribe|>{ts}<|endoftext|>"

    result.append({
        'new_text': new_text,
        'audio_filename': audio,
    })

In [7]:
len(result)

671

In [8]:
result[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.42|> Isi tempat kosong dengan jawapan yang paling sesuai.<|3.18|><|3.72|> Pengunjung yang berkelah di tepi sungai itu terkejut dengan kehadiran beberapa ekor ,<|7.96|><|8.18|> Tempat Kosong , yang tenggelam timbul di permukaan sungai.<|10.96|><|11.64|> A. harimau berantai.<|13.38|><|13.64|> B. buaya tembaga.<|15.66|><|16.36|> C.<|16.78|><|17.12|> bunga dedap.<|17.96|><|18.24|> D. buaya darat.<|19.36|><|endoftext|>',
 'audio_filename': 'tatabahasa-v2/0.mp3'}

In [9]:
import pandas as pd

pd.DataFrame(result).to_parquet('tatabahasa-segment-timestamps.parquet')

In [10]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="tatabahasa-segment-timestamps.parquet",
    path_in_repo="data/tatabahasa_segment-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

tatabahasa-segment-timestamps.parquet:   0%|          | 0.00/122k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/ac890967abbc31ef3dee1ba7b985ec8dbf208b9f', commit_message='Upload data/tatabahasa_segment-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='ac890967abbc31ef3dee1ba7b985ec8dbf208b9f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)